In [ ]:
#Compute effective T 
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

phi = torch.tanh
dt = 0.1
steps = 3000
num_trials = 200
steady_check_interval = 10   
steady_threshold = 1e-5      
avg_window = 40              

def check_steady(kinetic_history,window=3,threshold=1e-5):
    if len(kinetic_history) < window *2 :
        return False
    recent = np.mean(kinetic_history[-window:])
    pre = np.mean(kinetic_history[-2*window+1:-window])
    return abs(recent - pre) < threshold

def Classical_rnn_energy(J, num_trials=num_trials):
    kinetic_list = []
    for _ in range(num_trials):
        x = torch.randn(N, device=device)
        kinetic_history = []
        steady_reached = False
        buffer = []  
        for t in range(steps):
            dx = -x + J @ phi(x)
            x = x + dt * dx
            buffer.append((dx**2).mean().item()/2)
            if t>1000 and (t + 1) % steady_check_interval == 0:
                mean_kin = np.mean(buffer[-avg_window:])
                kinetic_history.append(mean_kin)
                if check_steady(kinetic_history, threshold=steady_threshold):
                    steady_reached = True
                    break
        if steady_reached:
            for _ in range(200):
                dx = -x + J @ phi(x)
                x = x + dt * dx
                kinetic_list.append((dx**2).mean().item()/2)
    return np.mean(kinetic_list)

def Langevin_dynamics_energy(J, T, num_trials=num_trials):
    kinetic_list = []
    for _ in range(num_trials):
        x = torch.randn(N, device=device)
        kinetic_history = []
        steady_reached = False
        buffer = []
        for t in range(steps):
            x = x.detach().requires_grad_(True)
            inner = -x + J @ phi(x)
            E = 0.5 * (inner**2).sum()
            gradE, = torch.autograd.grad(E, x)
            noise = torch.sqrt(torch.tensor(2*T*dt, device=device)) * torch.randn_like(x)
            dx = -gradE * dt + noise
            x = x + dx
            buffer.append((dx**2).mean().item()/2)
            if t>1000 and (t + 1) % steady_check_interval == 0:
                mean_kin = np.mean(buffer[-avg_window:])
                kinetic_history.append(mean_kin)
                if check_steady(kinetic_history, threshold=steady_threshold):
                    steady_reached = True
                    break
        if steady_reached:
            for _ in range(200):
                x = x.detach().requires_grad_(True)
                inner = -x + J @ phi(x)
                E = 0.5 * (inner**2).sum()
                gradE, = torch.autograd.grad(E, x)
                noise = torch.sqrt(torch.tensor(2*T*dt, device=device)) * torch.randn_like(x)
                dx = -gradE * dt + noise
                x = x + dx
                kinetic_list.append((dx**2).mean().item()/2)
    return np.mean(kinetic_list)

def find_effective_temperature(J, target_energy, T_low=0.05, T_high=0.2, tol=1e-6, max_iter=20):
    for i in range(max_iter):
        T_mid = 0.5 * (T_low + T_high)
        E_mid = Langevin_dynamics_energy(J, T_mid)
        print(f"Iter {i+1}: T_mid={T_mid:.6f}, E_mid={E_mid:.6f}")
        if abs(E_mid - target_energy) < tol:
            return T_mid, E_mid
        if E_mid > target_energy:
            T_high = T_mid
        else:
            T_low = T_mid
    return T_mid, E_mid

N = 1000             
g = 1.5               
torch.manual_seed(1997)
N_List = [1000,5000,10000]
Teff = []
for N in N_List:
    J = torch.randn(N, N, device=device) * g / np.sqrt(N)
    E_rnn = Classical_rnn_energy(J, num_trials=num_trials)
    print(f"Average kinetic energy (classical RNN): {E_rnn:.5f}")
    T_eff, E_eff = find_effective_temperature(J, E_rnn)
    print(f"\nEstimated effective temperature: T_eff = {T_eff:.6f}, E = {E_eff:.6f}")
    Teff.append(T_eff)

df = pd.DataFrame({ "N":N_List, "T_eff": Teff})
df.to_csv("Teff.csv", index=False)
print("✅ Data saved to energy_vs_T.csv")
with open("E_rnn_value.txt", "w") as f:
    f.write(str(E_rnn))
